# LLM generation text detection 

In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
import keras_nlp
import re
import spacy
import time
import string


train_essays = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv")
train_prompts = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv")
train_daigt = pd.read_csv("/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv")




## Datasets 

In [10]:
train_daigt["generated"] = train_daigt["label"]
train_essays = train_essays.merge(train_prompts, on='prompt_id', how='inner')

train_essays = pd.concat([train_essays,train_daigt])

#train_essays.loc[:,["prompt_name","generated"]].value_counts()
train_essays = train_essays.sample(frac = 1)

train_essays = train_essays.loc[:,["text","generated"]]

train_essays = train_essays.iloc[:10]

## Preprocessing

In [11]:
# Load the English NLP model
nlp = spacy.load("en_core_web_sm")

table = str.maketrans("", "", string.punctuation)
table[10] = None #\n
table[92] = None #\


#to lower
for code in range(26):
    table[code + 65] = code +97




def preprocess_text(text):
    
    #characters level preprocessing
    #remove \n and \, remove puntuactions, to lower case
    text = text.translate(table)
    
    # Tokenization using spaCy
    doc = nlp(text)
    
    # Remove stopwords and lemmization using spaCy's built-in stopword list
    tokens = [token.lemma_ for token in doc if not nlp.vocab[token.text].is_stop]
    
    return " ".join(tokens)




start_time = time.time()
train_essays.loc[:,"text"] = train_essays.loc[:,"text"].map(lambda x : preprocess_text(x))
# Convert back to pandas DataFrame (if needed)
end_time = time.time()

print(end_time - start_time)

1.0530376434326172


## BERT

In [12]:
import tensorflow as tf

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

X = train_essays["text"].values
y = train_essays["generated"].values



#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Assuming your labels are 0 and 1
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)


# Backbone
backbone = keras_nlp.models.BertBackbone.from_preset('bert_large_en_uncased')


# Preprocessor
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_en_uncased")



train_preprocessed = (
    imdb_train.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)
test_preprocessed = (
    imdb_test.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

backbone.trainable = False
inputs = backbone.input
sequence = backbone(inputs)["pooled_output"]

outputs = keras.layers.Dense(2)(sequence[:, backbone.cls_token_index, :])
x = keras.layers.Dense(hidden_size)(x)
x = keras.layers.Dense(hidden_size, activation = 'relu')(x)
x = keras.layers.Dropout(dropout_prob)(x)
x = keras.layers.Dense(hidden_size, activation='relu'),
x = keras.layers.Dense(hidden_size,  activation='relu'),
x = keras.layers.Dropout(dropout_prob),
x = keras.layers.Dense(hidden_size//8, activation = 'relu'),
outputs = keras.layers.Dense(1, activation='sigmoid')
model = keras.Model(inputs, outputs)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.AdamW(5e-5),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    jit_compile=True,
)
model.summary()
model.fit(
    imdb_train_preprocessed,
    validation_data=imdb_test_preprocessed,
    epochs=3,
)




hidden_size = 512
dropout_prob = 0.2
classifier_layers = [
    keras.layers.Dense(hidden_size),
    keras.layers.Dense(hidden_size, activation = 'relu'),
    keras.layers.Dropout(dropout_prob),
    keras.layers.Dense(hidden_size, activation='relu'),
    keras.layers.Dense(hidden_size,  activation='relu'),
    keras.layers.Dropout(dropout_prob),
    keras.layers.Dense(hidden_size//8, activation = 'relu'),
    keras.layers.Dense(1, activation='sigmoid')

]

backbone.trainable = False

inputs = keras.Input(shape=(None,), dtype=tf.string)

preprocess_input = preprocessor(inputs)

bert_output = backbone(preprocess_input)  # Get the output of the Bert model


outputs = classifier_layers(bert_output)  # Add the classification layers
model = tf.keras.Model(inputs=inputs, outputs=outputs)








    


Attaching 'config.json' from model 'keras/bert/keras/bert_large_en_uncased/1' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/bert/keras/bert_large_en_uncased/1' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/bert/keras/bert_large_en_uncased/1' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/bert/keras/bert_base_en_uncased/1' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/bert/keras/bert_base_en_uncased/1' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.txt' from model 'keras/bert/keras/bert_base_en_uncased/1' to your Kaggle notebook...


AttributeError: Exception encountered when calling layer "bert_preprocessor_1" (type BertPreprocessor).

in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras_nlp/src/models/bert/bert_preprocessor.py", line 161, in call  *
        token_ids, segment_ids = self.packer(x)
    File "/opt/conda/lib/python3.10/site-packages/keras_nlp/src/layers/preprocessing/preprocessing_layer.py", line 43, in __call__  *
        outputs = super().__call__(*args, **kwargs)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filevu4quav4.py", line 10, in tf__call
        (inputs, unbatched) = ag__.converted_call(ag__.ld(self)._sanitize_inputs, (ag__.ld(inputs),), None, fscope)
    File "/tmp/__autograph_generated_filegr8zcj9s.py", line 40, in tf___sanitize_inputs
        (inputs, unbatched_list, _) = ag__.converted_call(ag__.ld(list), (a

In [ ]:
#input
#backbone
#dropout
#logits(dense) with sigmoid

layers = classifier.layers
for layer in layers[:]:
    print(layer.name)

## Training

In [ ]:

learning_rate = 4e-3
batch_size = 16

#grid search



classifier.compile(
    loss = keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=[keras.metrics.AUC()],
    optimizer = keras.optimizers.Adam(learning_rate),
    jit_compile = True,
)



history = classifier.fit(
    x = X,
    y = y,
    #validation_data = (X_val, y_val),
    batch_size = batch_size,
    epochs = 10,  # Set the number of epochs as required
    class_weight = {0:class_weights[0],1:class_weights[1]}
)

## Evaluation and submission

In [ ]:
test_essays = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")
test_essays.loc[:,"text"] = test_essays.loc[:,"text"].map(lambda x : preprocess_text(x))

submission_dict  = pd.DataFrame()
submission_dict["id"] = test_essays["id"]
submission_dict["generated"] = np.around(classifier.predict(test_essays["text"])[:,0],1)


submission = pd.DataFrame.from_dict(submission_dict)
submission.to_csv("/kaggle/working/submission.csv",index=False)